<a href="https://colab.research.google.com/github/ThomasWong2022/ThomasWong2022.github.io/blob/main/Bond_Pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
"""


Bond Yield


"""

import scipy
import datetime


def years_to_maturity(maturity_date, ref_date=None):
    end = datetime.datetime.strptime(maturity_date, "%Y-%m-%d")
    if not ref_date:
        start = datetime.datetime.now()
    else:
        start = datetime.datetime.strptime(ref_date, "%Y-%m-%d")
    return (end - start).days / 366


def bond_yield(current, coupon_rate=0.125, freq=2, ref_date=None, maturity_date="2026-01-30", par=100):

    maturity = years_to_maturity(maturity_date,ref_date)

    def bond_formula(d, current, par, coupon_rate=0.125, freq=2, maturity=2.75):
        coupon = coupon_rate / freq
        cashflow_count = int(np.floor(maturity / (1 / freq)))
        second_cashflow_day = maturity - cashflow_count / freq + 1 / freq
        bond_price = par * np.power(d, maturity) + coupon * (
            np.power(d, maturity + 1 / freq) - np.power(d, second_cashflow_day)
        ) / (np.power(d, 1 / freq) - 1)
        accured_int = coupon_rate * (1/freq - second_cashflow_day)
        return bond_price - current

    coupon_rate_guess = (
        coupon_rate + np.exp(np.log(par / current) / maturity) - 1
    ) / 100
    x0 = 1 / (1 + coupon_rate_guess)
    ans = scipy.optimize.fsolve(
        bond_formula, x0=x0, args=(current, par, coupon_rate, freq, maturity)
    )
    return 1 / ans - 1

In [ ]:
bond_yield(30.17,coupon_rate=0.5,freq=2,maturity_date='2061-01-31',par=100) * 100

array([4.30427542])

In [ ]:
bond_yield(39.665,coupon_rate=0.625,freq=2,maturity_date='2050-11-20',par=100) * 100

array([4.59296366])

In [ ]:
bond_yield(30.17,coupon_rate=0.4,freq=2,maturity_date='2061-01-31',par=100) * 100

array([4.09790288])

In [ ]:
X = bond_yield(75.175,coupon_rate=0.125,freq=2,maturity_date='2051-03-23',par=100) * 100
X

array([1.20956811])

In [ ]:
! pip install numerapi

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from numerapi import NumerAPI
napi = NumerAPI()

In [ ]:
napi.download_dataset(
	"v4/validation.parquet",
	"validation.parquet"
)

validation.parquet: 1.31GB [00:46, 28.2MB/s]                            


'validation.parquet'

In [ ]:
napi.download_dataset(
	"v4/validation_example_preds.parquet",
	"validation_example_preds.parquet"
)

validation_example_preds.parquet: 64.2MB [00:02, 23.4MB/s]                            


'validation_example_preds.parquet'

In [ ]:
napi.download_dataset(
	"v4.3/validation_benchmark_models.parquet",
	"validation_benchmark_models.parquet"
)

validation_benchmark_models.parquet: 828MB [00:33, 24.6MB/s]                           


'validation_benchmark_models.parquet'

In [ ]:
v4_data = pd.read_parquet('validation.parquet',columns=['era','target_nomi_v4_20'])

In [ ]:
v4_example_pred = pd.read_parquet("validation_example_preds.parquet")

In [ ]:
v4_data['prediction'] = v4_example_pred['prediction']

In [ ]:
import datetime, scipy

## Convert datetime into Numerai eras
def convert_datetime_to_era(sample_date):
    baseline = datetime.datetime(year=2003, month=1, day=3)
    differences = datetime.datetime.strptime(sample_date, "%Y-%m-%d") - baseline
    new_era = str(differences.days // 7 + 1)
    while len(new_era) < 4:
        new_era = "0" + new_era
    return new_era


def convert_era_to_datetime(era):
    baseline = datetime.datetime(year=2003, month=1, day=3)
    new_datetime = baseline + datetime.timedelta(days=7 * (int(era) - 1))
    return new_datetime


def strategy_metrics(
    strategy_raw, interval=1, compound=False, accuracy=4, no_days=20, payout_ratio=0.06
):
    strategy = np.array(strategy_raw)
    if len(strategy.shape) < 2:
        strategy = strategy.reshape(-1, 1)
    epsilon = 1e-6
    results = dict()
    results["mean"] = np.mean(strategy, axis=0) * interval
    results["volatility"] = np.clip(
        np.std(strategy, axis=0) * np.sqrt(interval), epsilon, np.inf
    )
    results["skew"] = scipy.stats.skew(strategy, axis=0)
    results["kurtosis"] = scipy.stats.kurtosis(strategy, axis=0)
    if not compound:
        portfolio = np.cumsum(strategy, axis=0)
        temp = pd.DataFrame(portfolio).cummax(axis=0).values
        dd = portfolio - temp
    else:
        portfolio = np.cumprod(1 + strategy * payout_ratio * 5, axis=0)
        temp = pd.DataFrame(portfolio).cummax(axis=0).values
        dd = (portfolio - temp) / temp
    if compound:
        portfolio_ts = pd.DataFrame(portfolio)
        log_returns = np.log(portfolio_ts) - np.log(portfolio_ts.shift(1))
        results["mean"] = np.mean(log_returns) * 52
        results["volatility"] = np.std(log_returns) * np.sqrt(52)
    results["max_drawdown"] = np.clip(-1 * dd.min(axis=0), epsilon, np.inf)
    results["sharpe"] = results["mean"] / results["volatility"]
    results["calmar"] = results["mean"] / results["max_drawdown"]
    df = pd.DataFrame(results)
    if isinstance(strategy_raw, pd.DataFrame):
        df.index = strategy_raw.columns
        return df.round(accuracy)
    else:
        ## For Backward Comptability with Optuna Optimisation
        return df.round(accuracy).loc[0].to_dict()

def numerai_corr(preds, target):
    ranked_preds = (preds.rank(method="average").values - 0.5) / preds.shape[0]
    gauss_ranked_preds = scipy.stats.norm.ppf(ranked_preds)
    # make targets centered around 0. This assumes the targets have a mean of 0.5
    centered_target = target - 0.5
    # raise both preds and target to the power of 1.5 to accentuate the tails
    preds_p15 = np.sign(gauss_ranked_preds) * np.abs(gauss_ranked_preds) ** 1.5
    target_p15 = np.sign(centered_target) * np.abs(centered_target) ** 1.5
    # finally return the Pearson correlation
    return np.corrcoef(
        np.array(preds_p15).astype(float), np.array(target_p15).astype(float)
    )[0,1]


def numerai_corr_old(preds, target):
    ranked_preds = (preds.rank(method="average").values - 0.5) / preds.shape[0]
    gauss_ranked_preds = scipy.stats.norm.ppf(ranked_preds)
    # make targets centered around 0. This assumes the targets have a mean of 0.5
    centered_target = target - 0.5
    # finally return the Pearson correlation
    return np.corrcoef(
        np.array(gauss_ranked_preds).astype(float), np.array(centered_target).astype(float)
    )[0,1]

In [ ]:
## Score Example Model from Era 646 to Era 1030


performances = dict()

for i,df in v4_data.groupby("era"):
    performances[i] = numerai_corr_old(df['prediction'],df['target_nomi_v4_20'])

performances_v4_example = pd.Series(performances)

In [ ]:
strategy_metrics(performances_v4_example.loc['0625':'1030'])

{'mean': 0.0332,
 'volatility': 0.0357,
 'skew': -0.0673,
 'kurtosis': 0.1754,
 'max_drawdown': 0.2693,
 'sharpe': 0.9292,
 'calmar': 0.1231}